In [1]:
"""experiment
Logic: train model: 1 epoch, training dataset: 4096, eval datasets obtained thru select random subset"""

'experiment\nLogic: train model: 1 epoch, training dataset: 4096, eval datasets obtained thru select random subset'

In [2]:
#get models
%load_ext autoreload
%autoreload 2

In [3]:
from src.data.data import get_in_domain, get_out_domain

in_domain_train, in_domain_test = get_in_domain()
out_domain = get_out_domain()

print(f"In domain:\n{in_domain_train}")
print(in_domain_train[0])

print(f"Out of domain:\n{out_domain}")
print(out_domain[10])

# get_random_subsets(in_domain)

In domain:
Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 261802
})
{'premise': 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him', 'hypothesis': 'You lose the things to the following level if the people recall.', 'label': 0, 'idx': 1}
Out of domain:
Dataset({
    features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
    num_rows: 10000
})
{'premise': 'The president avoided the athlete .', 'hypothesis': 'The athlete avoided the president .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NN president)) (VP (VBD avoided) (NP (DT the) (NN athlete))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN at

In [4]:
from src.data.data import get_random_subsets, get_context_distillation_datasets
from src.finetuners.context_distillation_recursive import batch_recursive_context_distillation
from src.finetuners.context_distillation import batch_context_distillation

train_datasets, eval_dataset_in, eval_dataset_out = get_random_subsets(train_dataset=in_domain_train, 
                                                                       eval_dataset_in=in_domain_test, 
                                                                       eval_dataset_out=out_domain, 
                                                                       train_sample_sizes=[16],
                                                                       num_trials=10,   # 5
                                                                       eval_sample_size=50)

context_distillation_datasets = get_context_distillation_datasets(dataset=in_domain_train,
                                                                  train_datasets=train_datasets,
                                                                  fewshot_sample_size=16,
                                                                  large_sample_size=4096)

batch_context_distillation(model_names=['opt-350m'],
                           in_domain_dataset=in_domain_train,
                           train_datasets=context_distillation_datasets,
                           eval_dataset_in=eval_dataset_in,
                           eval_dataset_out=eval_dataset_out,
                           batch_size=2,
                           exp_label='test')

c:\Users\harri\anaconda3\envs\fine-tuning\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


opt-350m 16-shot:   0%|          | 0/9 [00:00<?, ?it/s]

opt-350m 4096-shot:   0%|          | 0/2049 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from src.model.model import get_model, download_model

# Get student models
student_model_opt125_causal, tokenizer_opt125_causal = get_model(model_name='opt-125m', model_type='CausalLM', pretrained=True)
student_model_opt350_causal, tokenizer_opt350_causal = get_model(model_name='opt-350m', model_type='CausalLM', pretrained=True)

# Get teacher models
teacher_model_opt125_causal, tokenizer_opt125_causal = get_model(model_name='opt-125m', model_type='CausalLM', pretrained=True)
teacher_model_opt350_causal, tokenizer_opt350_causal = get_model(model_name='opt-350m', model_type='CausalLM', pretrained=True)

In [ ]:
import json
from src.finetuners.fewshot import batch_fine_tune
from src.data.data import get_random_subsets

# Generate training and evaluation datasets. These should be used for all fine-tuning methods to ensure consistency. np.random should be seeded before this.
train_datasets, eval_dataset_in, eval_dataset_out = get_random_subsets(train_dataset=in_domain_train, 
                                                                       eval_dataset_in=in_domain_test, 
                                                                       eval_dataset_out=out_domain, 
                                                                       train_sample_sizes=[4096],
                                                                       num_trials=10,   # 5
                                                                       eval_sample_size=50)

In [ ]:
teacher_model_opt125_causal.device

device(type='cuda', index=0)

In [ ]:
from src.finetuners.context_distillation import context_distillation
import pprint
print("opt 125")
metrics = context_distillation(
    student_model_opt125_causal,
    teacher_model_opt125_causal, 
    tokenizer_opt125_causal, 
    train_datasets[4096][0], 
    num_epochs = 1,
    eval_dataset_in=eval_dataset_in, 
    eval_dataset_out=eval_dataset_out, 
    batch_size=2,
    model_name='opt-125m')
print("Metrics:")
pprint.pprint(metrics)

opt 125


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In domain eval metrics:
{'loss': 0.6931471824645996, 'accuracy': 0.58, 'runtime': 37.45100116729736, 'samples_per_second': 1.3350777934252012, 'peak_memory_gb': 9.791648864746094}


  0%|          | 0/7 [00:00<?, ?it/s]

Out of domain eval metrics:
{'loss': 0.6931471824645996, 'accuracy': 0.4, 'runtime': 33.18849849700928, 'samples_per_second': 1.506546010344688, 'peak_memory_gb': 9.791648864746094}
Metrics:
{'eval_in_accuracy': 0.58,
 'eval_in_loss': 0.6931471824645996,
 'eval_in_peak_memory_gb': 9.791648864746094,
 'eval_in_runtime': 37.45100116729736,
 'eval_in_samples_per_second': 1.3350777934252012,
 'eval_out_accuracy': 0.4,
 'eval_out_loss': 0.6931471824645996,
 'eval_out_peak_memory_gb': 9.791648864746094,
 'eval_out_runtime': 33.18849849700928,
 'eval_out_samples_per_second': 1.506546010344688,
 'model_name': 'opt-125m'}


In [ ]:
from src.finetuners.context_distillation import context_distillation
import pprint
print("opt 350")
metrics = context_distillation(
    student_model_opt350_causal,
    teacher_model_opt350_causal, 
    tokenizer_opt350_causal, 
    train_datasets[4096][0], 
    num_epochs = 1,
    eval_dataset_in=eval_dataset_in, 
    eval_dataset_out=eval_dataset_out, 
    batch_size=2,
    model_name='opt-350m')
print("Metrics:")
pprint.pprint(metrics)

In [ ]:
from src.finetuners.context_distillation_recursive import recursive_context_distillation
import pprint
print("opt 125 recursive distillation")
metrics = recursive_context_distillation(
    student_model_opt125_causal,
    tokenizer_opt125_causal, 
    train_datasets[4096][0], 
    num_epochs = 1,
    eval_dataset_in=eval_dataset_in, 
    eval_dataset_out=eval_dataset_out, 
    batch_size=2,
    model_name='opt-125m')
print("Metrics:")
pprint.pprint(metrics)

opt 125 recursive distillation


c:\Users\Joel\anaconda3\envs\fine-tuning2\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2048 [00:00<?, ?it/s]

Filter:   0%|          | 0/4096 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4096 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
from src.finetuners.context_distillation_recursive import recursive_context_distillation
import pprint
print("opt 350 recursive distillation")
metrics = recursive_context_distillation(
    student_model_opt350_causal,
    tokenizer_opt350_causal, 
    train_datasets[4096][0], 
    num_epochs = 1,
    eval_dataset_in=eval_dataset_in, 
    eval_dataset_out=eval_dataset_out, 
    batch_size=2,
    model_name='opt-350m')
print("Metrics:")
pprint.pprint(metrics)

opt 350 recursive distillation


  0%|          | 0/2048 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 198.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Of the allocated memory 14.00 GiB is allocated by PyTorch, and 207.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF